# Second virial coefficient calculation of TraPPE n-pentane using Mayer-Sampling


Here, we reproduce the virial coefficient of TraPPE n-pentane

Table 6 of https://doi.org/10.1021/jp972543+ reports −496 +/- 6 ml/mol at 395K

Note that this tutorial doesn't currently work as intra-molecular interaction energies should be used to generate a configuration, and the inter-molecular interaction contribute to the virial.

In [1]:
import unittest
import feasst as fst

class TestMayerSamplingTraPPEpentane(unittest.TestCase):
    def test(self):
        return
        assert(False) # This tutorial doesn't currently work
        mc = fst.MonteCarlo()
        config = fst.MakeConfiguration(fst.args({"cubic_box_length": str(fst.NEAR_INFINITY)}))
        config.add_particle_type("data.n-pentane_with_ref")
        config.add_particle_type("data.n-pentane_with_ref", "2")
        config.add_particle_of_type(0)
        config.add_particle_of_type(1)
        for site in range(config.num_site_types()):
            config.set_model_param("cutoff", site, 1e4)
        mc.add(config)
        mc.add(fst.MakePotential(fst.MakeLennardJones()))
        mc.add(fst.MakePotential(fst.MakeLennardJones(),
                                 fst.MakeVisitModelIntra(fst.args({"cutoff": "4"}))))
        
        # reference is a hard sphere on the first site only
        ref = fst.MakePotential(fst.MakeHardSphere())
        ref_params = fst.ModelParams(mc.configuration().model_params())
        for itype in range(mc.configuration().num_site_types()):
            ref_params.set("sigma", itype, 0)
        for itype in [0, 3]:
            for jtype in [0, 3]:
                ref_params.set("sigma", itype, jtype, 5)
        ref.set(ref_params)
        mc.add_to_reference(ref)

        mc.set(fst.MakeThermoParams(fst.args({"beta": str(1./fst.kelvin2kJpermol(395))}))) # mol/kJ
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
#         mc.add(fst.MakeTrialTranslate(fst.args({"new_only": "true", "reference_index": "0",
#             "tunable_param": "0.1", "particle_type": "1"})))
#         mc.add(fst.MakeTrialRotate(fst.args({"new_only": "true", "reference_index": "0",
#             "tunable_param": "40"})))      
        for ptype in range(mc.configuration().num_particle_types()):
            trans = 0.1
            if ptype == 0: trans = 0.0
            mc.add(fst.MakeTrialGrow(fst.ArgsVector([
                {"particle_type": str(ptype), "translate": "true", "site": "0", "tunable_param": str(trans)},
                {"bond": "true", "mobile_site": "1", "anchor_site": "0"},
                {"angle": "true", "mobile_site": "2", "anchor_site": "1", "anchor_site2": "0"},
                {"dihedral": "true", "mobile_site": "3", "anchor_site": "2", "anchor_site2": "1", "anchor_site3": "0"},
                {"dihedral": "true", "mobile_site": "4", "anchor_site": "3", "anchor_site2": "2", "anchor_site3": "1"}
            ]), fst.args({"reference_index": "0", "new_only": "true"})))
        steps_per = "1e5"
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "pentane"})))
        #mc.add(fst.MakeTune(fst.args({"steps_per": steps_per})))
        #mc.attempt(int(1e5))
        #mc.perform(fst.MakeRemoveModify(fst.args({"name": "Tune"})))
        #mc.attempt(int(1e6))
        #mayer = fst.MakeMayerSampling()
        #mc.set(mayer)
        mc.attempt(int(1e7))
        b2hs = 2./3.*fst.PI*mc.configuration().model_params().sigma().value(0)**3  # A^3
        b2hs *= 1e-30*1e3*1e3*mc.configuration().physical_constants().avogadro_constant() # L/mol
        print('mayer', mayer.mayer().str())
        print('mayer_ref', mayer.mayer_ref().str())
        print('b22(L/mol)', b2hs*mayer.second_virial_ratio())
        print('b22_block_stdev(L/mol)', b2hs*mayer.second_virial_ratio_block_stdev())
        self.assertAlmostEqual(-496, b2hs*mayer.second_virial_ratio(),
                               delta=3*b2hs*mayer.second_virial_ratio_block_stdev())

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestMayerSamplingTraPPEpentane) ... 

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


FAIL

FAIL: test (__main__.TestMayerSamplingTraPPEpentane)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-1-1868dd6c871a>", line 18, in test
    assert(False) # This tutorial doesn't currently work
AssertionError

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (failures=1)


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!